# Silhouete Artifact

Artifact of the paper "Silhouette: Leveraging Consistency Mechanisms to Detect Bugs in Persistent Memory-Based File Systems" from USENIX FAST '25.

- Git Repo: https://github.com/iaoing/Silhouette
- Zenodo records: https://zenodo.org/records/14550794

To access ChameleonCloud resources, you may need the [account](https://chameleoncloud.readthedocs.io/en/latest/user/federation.html) to log in to ChameleonCloud. You also need to have a [project](https://chameleoncloud.readthedocs.io/en/latest/user/project.html) to allocate resources (e.g., node).

As artifact reviewers, you should have the account distributed or the project assigned by the FAST AE committee. If you do not have it, please contact the committee.

## 0. How to Use JupyterLab

If you are familier with ChameleonCloud's JuputerLab, you can skip this section.

If you are not familier with Chameleon Cloud, please refer to the [offical documentation](https://chameleoncloud.readthedocs.io/en/latest/index.html). 

#### 0.1 Execute a cell

![execute-a-cell](pics/silhouette_ae_0_1.png)

#### 0.2 Waiting for the execution to complete

![wait-complete](pics/silhouette_ae_0_2.png)

#### 0.3 Open a terminal

![oepn-termianl-1](pics/silhouette_ae_0_3_1.png)

![oepn-termianl-2](pics/silhouette_ae_0_3_2.png)

![oepn-termianl-3](pics/silhouette_ae_0_3_3.png)

#### 0.4 Get the floating IP (used to SSH to the server)

Pleae make sure the server is active and the connection is established.

![get-floating-ip](pics/silhouette_ae_0_4.png)

#### 0.5 Connect to the server (floating IP) in the terminal

Pleae make sure the server is active and the connection is established.

![log-into-server](pics/silhouette_ae_0_5.png)


#### 0.6 SCP file from the server to JupyterLab

- Open a new terminal (do not log in to the server)
- Execute the below commands
    ```shell
    # Assume the file path is /path/to/the/file
    scp cc@floating_ip:/path/to/the/file ./
    ```
- Then, click the refersh button. The scp-ed file will be shown in the list.

![scp-refresh](pics/silhouette_ae_0_6.png)

## 1. Prepare ChameleonCloud Server

Preparing ChameleonCloud server may take ~20 mins.

If any error occurs, please try to restart from Setp 1.1 and change the variables to a different name.

If the error repeats, please contact Chameleon for help, [help desk](https://chameleoncloud.org/user/help/).

#### 1.1 Set variables

In [1]:
import os
from datetime import timedelta
ae_lease_loc = "CHI@UC" # CHI@UC, CHI@TACC, etc.
ae_lease_name = f"{os.getenv('USER')}-fast25-silhouette-ae"
ae_lease_duration = timedelta(hours=24)

#### 1.2 Configuring resources

We need to configure the experiment "container," meaning to isolated, reproducible environment that our experiment will run in.

In [ ]:
from chi import context

# During the transition period, we need to opt into the some of the
# new python-chi functions. Otherwise the functional interface will
# return the old types.
context.version = "1.0"

context.choose_site(default=ae_lease_loc)
context.choose_project()

#### 1.3 Check available hardware

In [ ]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

#### 1.4 Reserve node

If the output shows `Lease {ae_lease_name} has reached status error`, please change the ae_lease_name (e.g., `ae_lease_name = f"{os.getenv('USER')}-fast25-silhouette-ae-2"`) and restart from the beginning.

In [ ]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(ae_lease_name, duration=ae_lease_duration)
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(show="widget", idempotent=True)

#### 1.5 Create a server on the node

Next, we will launch the reserved node with an image. We submit the request to create a server, using the reservation information from our lease. You'll need to wait for the server to fully provision, which can around **10-20 minutes** depending on the node.

If the output shows `Server has moved to status Error` or the server was not start after 20 mins, please modify the `ae_lease_loc` and `ae_lease_name`, then restart from the beginning. 

In [ ]:
from chi import server

my_server = server.Server(
    ae_lease_name,
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04",
)
my_server.submit(show="widget", idempotent=True)

#### 1.6 Configure networking

Need to configure the server to use the floating IP we reserved earlier. We'll also need to wait for the networking to finish configuring, which may take a few additional minutes.

In [ ]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

#### 1.7 Ready to use

In [ ]:
my_server.show("widget")

## 2. Prepare Codebase and VM

**NOTE**:
Forward commands to the server from Jupyter may have some unexpected issues (e.g., connection closed, server no response, timeout, web no response) and have to restart from allocating a new node and server, which is too time-consuming. 

Please refer to [Section 0.3](#3.-Open-a-terminal), [Section 0.4](#4.-Get-the-floating-IP-(used-to-SSH-to-the-server)), and [Section 0.5](#5.-Connect-to-the-server-(floating-IP)-in-the-terminal) to open a terminal and connect to the Chameleon server. Then, excute the below commands.


```shell
# 2.1 Clone Silhouette Repo
mkdir -p ~/silhouette_ae
cd ~/silhouette_ae
git clone https://github.com/iaoing/Silhouette.git

# 2.2 Download VM from Zenodo. The image is ~30GiB, downloading may take ~20 mins
mkdir -p ~/silhouette_ae/qemu_imgs
cd ~/silhouette_ae/qemu_imgs
wget https://zenodo.org/records/14550794/files/silhouette_guest_vm.qcow2

# 2.3 Install Deps and Prepare
cd /home/cc/silhouette_ae/Silhouette && bash ./install_dep.sh
cd /home/cc/silhouette_ae/Silhouette && bash ./prepare.sh
```


## 3. Reproduction

Please refer to [Section 0.3](#3.-Open-a-terminal), [Section 0.4](#4.-Get-the-floating-IP-(used-to-SSH-to-the-server)), and [Section 0.5](#5.-Connect-to-the-server-(floating-IP)-in-the-terminal) to open a terminal and connect to the Chameleon server. Then, excute the below commands.

#### 3.1 Reproduce Bugs

The bug reproduction may take ~2 hours because:
- Some bugs may block other bugs, we make separate tests for each one.
- Regardless of the number of test cases, Silhouette follows the same process to prepare virtual machines (VMs). As a result, over 90% of the time is spent setting up the VM for each bug reproduction.

Please refer to `evaluation/bugs/readme.md` in the git repo for the detailed instructions.

```shell
cd /home/cc/silhouette_ae/Silhouette/evaluation/bugs
nohup bash ./reproduce_all.sh &
```

When the test is done, please refer to `evaluation/bugs/bugXX/readme.md` in the git repo to check the output.

#### 3.2 Scalibility Evaluation

**3.2.1 Test NOVA on ACE-seq3 workload with 20 VMs**

Since testing NOVA, PMFS, and WineFS on ACE-seq3 workload with different crash plan generation schemes (Silhouette, 2CP, Invariants+Comb) may take more than 40 hours, we only test NOVA with Silhouette scheme generation here.

This test takes ~4-5 hours. 

Please refer to `evaluation/scalability/readme.md` for the detailed instructions.

```shell
cd /home/cc/silhouette_ae/Silhouette/evaluation/scalability/seq3/nova/mech2cp
nohup bash ./run.sh &
```

The result will be available in the `~/silhouette_ae/Silhouette/evaluation/scalability/seq3/nova/mech2cp/result` directory. please refer to the `Raw Result Layout` section in `evaluation/scalability/readme.md` to check the type generated result.

Samples:
```shell
# This shows the time breakdown of this test.
cat ~/silhouette_ae/Silhouette/evaluation/scalability/seq3/nova/mech2cp/result/result_elapsed_time/result_time.txt

# This shows the number of generated crash plans
cat ~/silhouette_ae/Silhouette/evaluation/scalability/seq3/nova/mech2cp/result/result_cps/result.txt
```

**3.2.2 Test ACE-seq2 workload**

Testing one file system on ACE-seq2 workload with one crash plan generation scheme takes ~30 mins. Therefore, we can run the entrie seq2 workload and plot Figrue 1 and Table 9, similar to that presented in the paper.

Figure 8 requires enabling certain bugs detected by Chipmunk and manual efforts to identify additional bugs, which may extend beyond the scope of this reproduction process.

Figure 9 contains parts of lines shown in Figure 1.

Since testing [Chipmunk](https://github.com/utsaslab/chipmunk) and [Vinter](https://github.com/KIT-OSGroup/vinter/tree/master) take longer time than Silhouette, and both tools have their artifacts available on GitHub, we will not discuss them further here.

```shell
# Run the test
cd ~/silhouette_ae/Silhouette/evaluation/scalability/seq2
nohup bash ./run_all.sh &

# Analyze the result and plot the figrue and table
cd ~/silhouette_ae/Silhouette/evaluation/scalability/seq2
bash ./plot.sh

# Check the table
cat ~/silhouette_ae/Silhouette/evaluation/scalability/seq2/table_9.txt

# Check the figure
# Please refer to Section 0.6 for how to `scp` files from the server to JupyterLab.
```